Wszystkie zadania są ważne do końca semestru, wszystkie zadania mają oznaczenie E. Lista nawiązuje do konkursu Zero Speech Challenge 2021 (ale w mocno uproszczonej postaci), w którym dysponując jedynie zbiorem nagrań mamy odpowiadać na różne pytania dotyczące języka. My skoncentrujemy się na jednym pytaniu: widząc dwa wyrazy zdecyduj, który z nich jest poprawnym słowem danego języka. Dodatkowo zrezygnujemy z aspektu związanego z dźwiękiem, czyli danymi będą (zniekształcone) teksty polskie. Plik definiujący zadanie składa się z par wyrazów (dla uproszczenia pierwszy wyraz zawsze jest poprawny, a drugi sztucznie utworzony, oczywiście z tej właściwości korzystamy tylko przy testowaniu). Rozwiązanie każdego z zadań powinno:

a) zawierać funkcję, która dla potencjalnego słowa podaje wartość punktową odpowiadającą przekonaniu programu, że jest to prawdziwe słowo (nie ma żadnych warunków na zakres tej wartości)

b) wykonać dostarczone testy, przypisując każdemu przypadkowi testowemu wartość 1, jeżeli poprawne słowo dostanie więcej punktów od niepoprawnego, 0.5 – jeżeli dostaną tyle samo, 0 – w
pozostałym przypadku.

c) Przedstawić uśredniony wynik testów2

Można korzystać z faktu, że średnia długość słowa to około 6.2 znaku. Każda dodatkowa wiedza o języku może pochodzić jedynie z danych dostarczonych wraz z zadaniem (oczywiście nie wolno wykorzystywać danych z zadania 1 do zadania 2 i odwrotnie).


Zadanie 1. (5p+) W tym zadaniu zniekształcenie polega na usunięciu spacji z każdego zdania (ale przy zachowaniu podziału na zdania, które ciągle są w osobnych linijkach). Zatem dane wyglądają tak:

poiiwojnieświatowejniebyłojużtejseriinapkp

poparuminutachtocośbyłojużbardzobliskoekranu

jestopartynazłymwilkuzczerwonegokapturkaitrzechmałychświnek

Pary słów-niesłów do tego zadania charakteryzują się tym, że:

a) słowa poprawne występują w oryginalnym tekście (ze spacjami)

b) słowa niepoprawne są ciągami liter z tekstu bez spacji (mamy dodatkowo gwarancję, że nie występowały one jako słowa w oryginalnym tekście, choć oczywiście mogą być przypadkowo poprawnymi słowami polskimi)

c) słowa niepoprawne dodatkowo zostały zaakceptowane przez prosty n-gramowy literkowy model językowy (przykładowo śbyłojużb zostałoby zapewne odrzucone)

Przykładowe testy:

zastanawiam taktymiędzy

drugorzędnie czegospowodo

przechowywania stanieprzekaza

doprowadził jarozwiązan

parkowanie zdzierasza

panowie wypełni

publicznych icznościami

ograniczyć ropejskiej


Do zaliczenia zadania trzeba osiągnąć poprawność 0.7 (aczkolwiek możliwe są częściowe punkty za mniejsze wyniki, uwzględniające nakład pracy studenta, wg uznania prowadzącego pracownię). Każde kolejne 0.04 to 1 punkt. W przypadku wyraźnego przekroczenia wartości 0.9 możliwe są dalsze premie egzaminacyjne, przydzielane podczas konsultacji z wykładowc

In [26]:
SENTENCES = []
with open('sentences_for_task1.txt', 'r', encoding='utf-8') as f:
    for i,line in enumerate(f):
        SENTENCES.append(line[:-1])

In [76]:
MAX_K = 5

In [77]:
PREFIXES = set()
SUFFIXES = set()

In [78]:
for sent in SENTENCES:
    for K in range(1, MAX_K):
        PREFIXES.add(sent[:K])
        SUFFIXES.add(sent[-K:])

In [79]:
WORDS = {}

def add_word(w):
    WORDS[w] = WORDS.get(w, 0) + 1

for sent in SENTENCES:
    for K in range(1, MAX_K):
        for i in range(len(sent)):
            pref = sent[i:i+K]
            if pref not in SUFFIXES:
                continue
            for j in range(i,i+8):
                suf = sent[j:j+K]
                if suf in SUFFIXES:
                    add_word(sent[i:j+K])

In [80]:
max_words_score = max(WORDS.values())

In [81]:
def score(w):
    words_score = WORDS.get(w, 0)/max_words_score
    prefix_score = 0
    suffix_score = 0
    for K in range(1, MAX_K):
        if w[:K] in PREFIXES:
            prefix_score += 1
        if w[-K:] in SUFFIXES:
            suffix_score += 1
    prefix_score /= MAX_K
    suffix_score /= MAX_K
    return words_score*0.2 + prefix_score*0.4 + suffix_score*0.4

In [82]:
final_score = 0
test_count = 0
with open('test_for_task1.txt', 'r', encoding='utf-8') as f:
    for i,line in enumerate(f):
        w1, w2 = line.split()
        s1, s2 = score(w1), score(w2)
        if s1>s2:
            final_score+=1
        elif s1==s2:
            final_score+=0.5
        test_count+=1

final_score/=test_count
final_score

0.71056